# Sports Betting Decision and Risk Mini Course
## Days 25-27: Portfolio Risk, Fair Games, and Value at Risk

In these three sessions we will explore:
- **Day 25**: Portfolio variance and correlation between bets
- **Day 26**: Martingales and fair games
- **Day 27**: Value at Risk (VaR) and worst case losses

We assume you know basic Python, numpy, matplotlib, and basic probability.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set random seed for reproducibility
np.random.seed(42)

# Plot styling
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

---
# Day 25: Portfolio Variance and Correlation

## Betting Portfolios

You can place several bets at once. Each bet has its own outcome. We call this collection a **portfolio**.

Each bet return is a random variable. The portfolio return is the sum of all individual bet returns.

## Variance, Covariance, and Correlation

**Variance** measures how much one bet return moves around its mean.

**Covariance** and **correlation** measure how two bet returns move together.
- **Positive correlation**: Bets tend to win together and lose together. This increases total portfolio risk.
- **Negative correlation**: Bets offset each other more often. This reduces total portfolio risk.
- **Zero correlation**: Bets move independently.

Correlation $\rho$ ranges from -1 to +1. It is related to covariance by:

$$\text{Cov}(R_1, R_2) = \rho \cdot \sigma_1 \cdot \sigma_2$$

where $\sigma_1$ and $\sigma_2$ are the standard deviations of the two returns.

## Portfolio Model with Two Bets

We define two bets with:
- Win probability $p_i$
- Decimal odds $O_i$
- Stake weight $w_i$ as a fraction of bankroll

Weights sum to 1 for the portfolio.

### Bet Return

For bet $i$:
- With probability $p_i$: return is $w_i \times (O_i - 1)$
- With probability $1 - p_i$: return is $-w_i$

### Portfolio Variance Formula

For two bets:

$$\text{Var}(R_{\text{port}}) = w_1^2 \text{Var}(R_1) + w_2^2 \text{Var}(R_2) + 2 w_1 w_2 \text{Cov}(R_1, R_2)$$

## Numeric Example: Two Correlated Bets

We compute portfolio variance for different correlation values.

In [ ]:
# Bet 1 parameters
p1 = 0.45  # Win probability
O1 = 2.2   # Decimal odds
w1 = 0.6   # Weight in portfolio

# Bet 2 parameters
p2 = 0.50
O2 = 2.0
w2 = 0.4

# Expected returns
E_R1 = p1 * w1 * (O1 - 1) + (1 - p1) * (-w1)
E_R2 = p2 * w2 * (O2 - 1) + (1 - p2) * (-w2)

print(f"Bet 1 expected return: {E_R1:.4f}")
print(f"Bet 2 expected return: {E_R2:.4f}")
print(f"Portfolio expected return: {E_R1 + E_R2:.4f}")

In [ ]:
# Variances of individual bets
# Var(R) = E[R^2] - E[R]^2

# For bet 1
win_return_1 = w1 * (O1 - 1)
lose_return_1 = -w1
E_R1_squared = p1 * win_return_1**2 + (1 - p1) * lose_return_1**2
Var_R1 = E_R1_squared - E_R1**2
sigma_1 = np.sqrt(Var_R1)

# For bet 2
win_return_2 = w2 * (O2 - 1)
lose_return_2 = -w2
E_R2_squared = p2 * win_return_2**2 + (1 - p2) * lose_return_2**2
Var_R2 = E_R2_squared - E_R2**2
sigma_2 = np.sqrt(Var_R2)

print(f"Bet 1 variance: {Var_R1:.4f}, std dev: {sigma_1:.4f}")
print(f"Bet 2 variance: {Var_R2:.4f}, std dev: {sigma_2:.4f}")

In [ ]:
# Portfolio variance for different correlation values

rho_values = [-0.5, 0.0, 0.5, 1.0]

print("Portfolio variance for different correlations:")
print("="*50)

for rho in rho_values:
    # Covariance from correlation
    Cov_R1_R2 = rho * sigma_1 * sigma_2
    
    # Portfolio variance
    Var_port = w1**2 * Var_R1 + w2**2 * Var_R2 + 2 * w1 * w2 * Cov_R1_R2
    sigma_port = np.sqrt(Var_port)
    
    print(f"Correlation ρ = {rho:5.1f}: Var = {Var_port:.4f}, Std Dev = {sigma_port:.4f}")

**Observation**: Higher correlation increases portfolio variance when both bets have positive weights. This is because bets tend to lose together more often.

---
## Exercise: Portfolio Variance with Multiple Correlated Bets

We verify the analytic formula by simulation and explore how correlation affects total risk.

In [ ]:
# Define three bets
bets = [
    {'p': 0.45, 'O': 2.2, 'w': 0.4},  # Bet 1
    {'p': 0.50, 'O': 2.0, 'w': 0.35}, # Bet 2
    {'p': 0.40, 'O': 2.5, 'w': 0.25}, # Bet 3
]

# Compute expected return and variance for each bet
n_bets = len(bets)
E_returns = []
variances = []
std_devs = []

for i, bet in enumerate(bets):
    p, O, w = bet['p'], bet['O'], bet['w']
    
    # Expected return
    E_R = p * w * (O - 1) + (1 - p) * (-w)
    
    # Variance
    win_ret = w * (O - 1)
    lose_ret = -w
    E_R_sq = p * win_ret**2 + (1 - p) * lose_ret**2
    Var_R = E_R_sq - E_R**2
    
    E_returns.append(E_R)
    variances.append(Var_R)
    std_devs.append(np.sqrt(Var_R))
    
    print(f"Bet {i+1}: E[R] = {E_R:.4f}, Var(R) = {Var_R:.4f}, σ = {np.sqrt(Var_R):.4f}")

print(f"\nPortfolio expected return: {sum(E_returns):.4f}")

In [ ]:
def compute_portfolio_variance_analytic(bets, correlation_matrix):
    """
    Compute portfolio variance analytically.
    
    Args:
        bets: list of bet dictionaries with 'p', 'O', 'w'
        correlation_matrix: n x n correlation matrix
    
    Returns:
        Portfolio variance
    """
    n = len(bets)
    
    # Compute individual variances and std devs
    vars_list = []
    stds_list = []
    weights = []
    
    for bet in bets:
        p, O, w = bet['p'], bet['O'], bet['w']
        weights.append(w)
        
        E_R = p * w * (O - 1) + (1 - p) * (-w)
        win_ret = w * (O - 1)
        lose_ret = -w
        E_R_sq = p * win_ret**2 + (1 - p) * lose_ret**2
        var = E_R_sq - E_R**2
        
        vars_list.append(var)
        stds_list.append(np.sqrt(var))
    
    # Build covariance matrix
    cov_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                cov_matrix[i, j] = vars_list[i]
            else:
                cov_matrix[i, j] = correlation_matrix[i, j] * stds_list[i] * stds_list[j]
    
    # Portfolio variance: w^T * Cov * w
    w_array = np.array(weights)
    var_portfolio = w_array @ cov_matrix @ w_array
    
    return var_portfolio

In [ ]:
def simulate_correlated_bets(bets, correlation_matrix, n_sims=100000):
    """
    Simulate correlated bet returns using a factor model.
    
    We use: R_i = sqrt(rho) * F + sqrt(1 - rho) * epsilon_i
    where F is a common factor and epsilon_i are independent noise terms.
    
    For simplicity with multiple bets, we use a single correlation parameter.
    """
    n_bets = len(bets)
    portfolio_returns = []
    
    # Extract common correlation (assume all pairs have same correlation for simplicity)
    # For a more general case, use Cholesky decomposition
    rho = correlation_matrix[0, 1] if n_bets > 1 else 0
    
    for _ in range(n_sims):
        # Common factor (affects all bets)
        common_factor = np.random.randn()
        
        total_return = 0
        
        for bet in bets:
            p, O, w = bet['p'], bet['O'], bet['w']
            
            # Create correlated random variable
            if rho != 0:
                idiosyncratic = np.random.randn()
                z = np.sqrt(abs(rho)) * common_factor + np.sqrt(1 - abs(rho)) * idiosyncratic
            else:
                z = np.random.randn()
            
            # Convert to win/loss using normal CDF approximation
            from scipy import stats
            # Threshold for win based on win probability
            threshold = stats.norm.ppf(1 - p)
            
            if z < threshold:  # Win
                bet_return = w * (O - 1)
            else:  # Loss
                bet_return = -w
            
            total_return += bet_return
        
        portfolio_returns.append(total_return)
    
    return np.array(portfolio_returns)

In [ ]:
# Test different correlation values
test_rhos = [0.0, 0.3, 0.5]

print("Comparison: Analytic vs Simulated Portfolio Variance")
print("="*60)

for rho in test_rhos:
    # Build correlation matrix (all pairs have same correlation)
    corr_matrix = np.ones((n_bets, n_bets)) * rho
    np.fill_diagonal(corr_matrix, 1.0)
    
    # Analytic variance
    var_analytic = compute_portfolio_variance_analytic(bets, corr_matrix)
    std_analytic = np.sqrt(var_analytic)
    
    # Simulated variance
    returns = simulate_correlated_bets(bets, corr_matrix, n_sims=100000)
    var_simulated = np.var(returns)
    std_simulated = np.std(returns)
    
    print(f"\nCorrelation ρ = {rho:.1f}")
    print(f"  Analytic:   Var = {var_analytic:.6f}, Std Dev = {std_analytic:.6f}")
    print(f"  Simulated:  Var = {var_simulated:.6f}, Std Dev = {std_simulated:.6f}")
    print(f"  Difference: {abs(var_analytic - var_simulated):.6f}")

In [ ]:
# Plot portfolio standard deviation as a function of correlation

rho_range = np.linspace(-0.5, 1.0, 30)
std_devs_list = []

for rho in rho_range:
    corr_matrix = np.ones((n_bets, n_bets)) * rho
    np.fill_diagonal(corr_matrix, 1.0)
    
    var_port = compute_portfolio_variance_analytic(bets, corr_matrix)
    std_devs_list.append(np.sqrt(var_port))

plt.figure(figsize=(10, 6))
plt.plot(rho_range, std_devs_list, linewidth=2.5, color='darkblue')
plt.xlabel('Correlation ρ', fontsize=12)
plt.ylabel('Portfolio Standard Deviation', fontsize=12)
plt.title('Portfolio Risk vs Correlation Between Bets', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("When correlation increases, portfolio standard deviation increases.")
print("This happens because bets tend to win and lose together.")
print("Negative correlation provides diversification and reduces total risk.")

---
# Day 26: Martingales and Fair Games

## Fair Games

A **fair game** means the expected change in bankroll is zero at each step.

Think of a fair coin flip:
- Heads wins 1 unit
- Tails loses 1 unit
- Probability of heads is 0.5

Expected gain per flip = $0.5 \times 1 + 0.5 \times (-1) = 0$.

## Martingales

A **martingale** is a bankroll process where the expected bankroll at the next step equals the current bankroll, given everything that happened so far.

In simple terms:

$$E[\text{Bankroll}_{t+1} \mid \text{History up to } t] = \text{Bankroll}_t$$

Fair games create martingales. Your expected bankroll stays flat even though actual paths wander.

## Fair Coin Flip Simulation

We simulate a fair coin flip game and track bankroll over many flips.

In [ ]:
def simulate_coin_flip_bankroll(initial_bankroll, n_flips, p_win, stake):
    """
    Simulate bankroll for a sequence of coin flips.
    
    Args:
        initial_bankroll: Starting bankroll
        n_flips: Number of flips
        p_win: Probability of winning each flip
        stake: Amount bet on each flip
    
    Returns:
        Array of bankroll values (length n_flips + 1)
    """
    bankroll = np.zeros(n_flips + 1)
    bankroll[0] = initial_bankroll
    
    for i in range(n_flips):
        # Flip coin
        if np.random.rand() < p_win:
            # Win
            bankroll[i+1] = bankroll[i] + stake
        else:
            # Lose
            bankroll[i+1] = bankroll[i] - stake
    
    return bankroll

In [ ]:
# Simulate fair game (p = 0.5)

initial_bankroll = 100
n_flips = 500
stake = 1
n_paths = 5

plt.figure(figsize=(12, 6))

for i in range(n_paths):
    bankroll_path = simulate_coin_flip_bankroll(initial_bankroll, n_flips, p_win=0.5, stake=stake)
    plt.plot(bankroll_path, alpha=0.7, linewidth=1.5)

plt.axhline(y=initial_bankroll, color='black', linestyle='--', linewidth=2, label='Starting Bankroll')
plt.xlabel('Flip Number', fontsize=12)
plt.ylabel('Bankroll', fontsize=12)
plt.title('Fair Game: p = 0.5 (Martingale)', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Observation: Paths wander randomly around the starting bankroll.")
print("Expected bankroll remains constant at each step.")

## Adding a Small Edge

Now we change the win probability to 0.52. This breaks fairness and gives positive expected gain per flip.

In [ ]:
# Compute expected gain per flip for p = 0.52

p_edge = 0.52
expected_gain = p_edge * stake + (1 - p_edge) * (-stake)

print(f"Win probability: {p_edge}")
print(f"Expected gain per flip: {expected_gain:.4f} units")
print(f"Expected bankroll after {n_flips} flips: {initial_bankroll + n_flips * expected_gain:.2f}")

In [ ]:
# Simulate game with edge

plt.figure(figsize=(12, 6))

for i in range(n_paths):
    bankroll_path = simulate_coin_flip_bankroll(initial_bankroll, n_flips, p_win=p_edge, stake=stake)
    plt.plot(bankroll_path, alpha=0.7, linewidth=1.5)

# Expected path
expected_path = initial_bankroll + np.arange(n_flips + 1) * expected_gain
plt.plot(expected_path, color='black', linestyle='--', linewidth=2.5, label='Expected Bankroll')

plt.xlabel('Flip Number', fontsize=12)
plt.ylabel('Bankroll', fontsize=12)
plt.title('Game with Edge: p = 0.52 (Positive Drift)', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Observation: Paths tend to drift upward.")
print("Expected bankroll grows linearly with each flip.")

---
## Exercise: Fair Game vs Game with Edge

We simulate many paths and compute average bankroll over time.

In [ ]:
# Simulate many paths for both fair and edge games

n_sims = 1000
n_flips = 500
initial_bankroll = 100
stake = 1

# Fair game
fair_paths = np.zeros((n_sims, n_flips + 1))
for i in range(n_sims):
    fair_paths[i, :] = simulate_coin_flip_bankroll(initial_bankroll, n_flips, p_win=0.5, stake=stake)

# Game with edge
edge_paths = np.zeros((n_sims, n_flips + 1))
for i in range(n_sims):
    edge_paths[i, :] = simulate_coin_flip_bankroll(initial_bankroll, n_flips, p_win=0.52, stake=stake)

# Compute average bankroll at each flip
avg_fair = np.mean(fair_paths, axis=0)
avg_edge = np.mean(edge_paths, axis=0)

In [ ]:
# Plot sample paths and average paths

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Fair game
for i in range(10):
    axes[0].plot(fair_paths[i, :], alpha=0.3, color='steelblue', linewidth=1)
axes[0].plot(avg_fair, color='darkblue', linewidth=3, label='Average Bankroll')
axes[0].axhline(y=initial_bankroll, color='black', linestyle='--', linewidth=2, alpha=0.5)
axes[0].set_xlabel('Flip Number', fontsize=12)
axes[0].set_ylabel('Bankroll', fontsize=12)
axes[0].set_title('Fair Game (p = 0.5)', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Game with edge
for i in range(10):
    axes[1].plot(edge_paths[i, :], alpha=0.3, color='coral', linewidth=1)
axes[1].plot(avg_edge, color='darkred', linewidth=3, label='Average Bankroll')
axes[1].axhline(y=initial_bankroll, color='black', linestyle='--', linewidth=2, alpha=0.5)
axes[1].set_xlabel('Flip Number', fontsize=12)
axes[1].set_ylabel('Bankroll', fontsize=12)
axes[1].set_title('Game with Edge (p = 0.52)', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**Summary:**

- **Fair game (p = 0.5)**: The average bankroll stays flat. This is a martingale. Individual paths wander but have no upward or downward trend.

- **Game with edge (p = 0.52)**: The average bankroll grows. This breaks the martingale property. Individual paths still wander but drift upward on average.

---
# Day 27: Value at Risk (VaR)

## What is Value at Risk?

**Value at Risk (VaR)** is a threshold loss that you are unlikely to exceed under a chosen probability level and time horizon.

**Example**: A 5% one-season VaR of 40 units means that in only 5% of seasons you lose more than 40 units.

VaR helps answer: "What is my worst case loss with reasonable confidence?"

## Link to Simulations

Given a distribution of final bankrolls or returns from simulations, we read VaR from a low percentile.

For a 5% VaR:
- Compute the 5th percentile of losses.
- This tells us the loss level that is exceeded only 5% of the time.

## Maximum Drawdown

**Drawdown** measures the largest peak-to-trough decline in bankroll during a period.

Steps to compute maximum drawdown:
1. Track running peak bankroll.
2. At each step, compute peak minus current bankroll.
3. Maximum drawdown is the largest such drop within the season.

We will estimate the 5% worst case drawdown from simulations.

---
## Exercise: Estimate 5% Worst Case Drawdown

We simulate many betting seasons and compute maximum drawdown for each.

In [ ]:
def compute_max_drawdown(bankroll_path):
    """
    Compute maximum drawdown from a bankroll path.
    
    Args:
        bankroll_path: Array of bankroll values over time
    
    Returns:
        Maximum drawdown (positive number)
    """
    running_peak = np.maximum.accumulate(bankroll_path)
    drawdown = running_peak - bankroll_path
    max_drawdown = np.max(drawdown)
    return max_drawdown

In [ ]:
# Simulate many seasons with a simple betting strategy
# Use fixed stake with a small edge

n_seasons = 10000
n_bets_per_season = 500
initial_bankroll = 100
p_win = 0.52  # Small edge
stake = 1     # Fixed stake per bet

max_drawdowns = []

for season in range(n_seasons):
    bankroll_path = simulate_coin_flip_bankroll(initial_bankroll, n_bets_per_season, p_win, stake)
    max_dd = compute_max_drawdown(bankroll_path)
    max_drawdowns.append(max_dd)

max_drawdowns = np.array(max_drawdowns)

In [ ]:
# Compute drawdown statistics

mean_drawdown = np.mean(max_drawdowns)
median_drawdown = np.median(max_drawdowns)

# 5% worst case: 95th percentile of drawdown distribution
var_5pct = np.percentile(max_drawdowns, 95)

print("Maximum Drawdown Statistics")
print("="*50)
print(f"Mean drawdown:          {mean_drawdown:.2f} units")
print(f"Median drawdown:        {median_drawdown:.2f} units")
print(f"5% worst case drawdown: {var_5pct:.2f} units")
print(f"\nInterpretation: In 95% of seasons, max drawdown stays below {var_5pct:.2f} units.")
print(f"                In only 5% of seasons, drawdown exceeds {var_5pct:.2f} units.")

In [ ]:
# Plot histogram of drawdowns

plt.figure(figsize=(12, 6))
plt.hist(max_drawdowns, bins=50, color='steelblue', alpha=0.7, edgecolor='black')
plt.axvline(x=var_5pct, color='red', linestyle='--', linewidth=3, label=f'5% VaR = {var_5pct:.2f}')
plt.axvline(x=mean_drawdown, color='green', linestyle='--', linewidth=2, label=f'Mean = {mean_drawdown:.2f}')
plt.axvline(x=median_drawdown, color='orange', linestyle='--', linewidth=2, label=f'Median = {median_drawdown:.2f}')

plt.xlabel('Maximum Drawdown (units)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Distribution of Maximum Drawdown over 10,000 Seasons', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

**Commentary:**

The 5% worst case drawdown tells us how bad things can get in the unlucky tail of outcomes.

This metric helps with:
- **Bankroll sizing**: You need enough capital to survive the 5% worst case without going broke.
- **Risk comfort**: Knowing the typical drawdown range helps set realistic expectations.
- **Strategy comparison**: Compare different betting strategies by their VaR profiles.

VaR does not tell you what happens beyond the 5% threshold. For that you might also look at expected shortfall or stress tests.

---
# Summary

Over these three days we covered:

## Day 25: Portfolio Variance and Correlation

- **Portfolio variance** depends on individual bet variances and the correlation between bets.
- **Higher correlation** increases total portfolio risk when bets have the same sign of exposure.
- **Negative correlation** provides diversification and reduces risk.
- We verified the analytic formula with simulations using a factor model.

## Day 26: Martingales and Fair Games

- A **fair game** has zero expected gain per step.
- Fair games create **martingales** where expected future bankroll equals current bankroll.
- Individual paths wander randomly even though the expected path is flat.
- Adding a small **edge** breaks fairness and creates positive drift in bankroll.

## Day 27: Value at Risk (VaR)

- **VaR** measures the threshold loss exceeded only with small probability (e.g., 5%).
- **Maximum drawdown** tracks the largest peak-to-trough decline in bankroll.
- The **5% worst case drawdown** is the 95th percentile of the drawdown distribution.
- VaR metrics help size bankroll and set realistic risk expectations.

## Practical Takeaways

1. **Diversification works** when bets are not perfectly correlated. Look for low or negative correlation.
2. **Fair games** may feel safe but offer no profit. You need an edge to grow bankroll.
3. **Simulate to understand risk**. Analytic formulas give quick answers but simulations show the full range of outcomes.
4. **Plan for worst cases**. Use VaR and drawdown analysis to avoid ruin in unlucky streaks.

These tools form the foundation for disciplined, risk-aware sports betting.